In [1]:
# importing packages & libraries
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# reading and processing data
data = open('irish-lyrics.txt').read()
corpus = data.lower().split("\n")

In [3]:
# defining tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# print(tokenizer.word_index)
print(total_words)

2690


In [4]:
# preprocessing data
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# padding sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# creating predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [5]:
# creating model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(LSTM(150)),
    tf.keras.layers.Dense(total_words, activation = 'softmax'),
])

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [6]:
# training model
history = model.fit(xs, ys, epochs=10, verbose=1)

Epoch 1/10
377/377 [==============================] - 30s 70ms/step - loss: 6.7303 - accuracy: 0.0652
Epoch 2/10
377/377 [==============================] - 26s 69ms/step - loss: 6.2314 - accuracy: 0.0781
Epoch 3/10
377/377 [==============================] - 26s 70ms/step - loss: 5.9626 - accuracy: 0.0860
Epoch 4/10
377/377 [==============================] - 27s 71ms/step - loss: 5.6671 - accuracy: 0.0999
Epoch 5/10
377/377 [==============================] - 27s 70ms/step - loss: 5.3404 - accuracy: 0.1141
Epoch 6/10
377/377 [==============================] - 26s 70ms/step - loss: 4.9927 - accuracy: 0.1318
Epoch 7/10
377/377 [==============================] - 27s 71ms/step - loss: 4.6350 - accuracy: 0.1497
Epoch 8/10
377/377 [==============================] - 27s 71ms/step - loss: 4.2716 - accuracy: 0.1784
Epoch 9/10
377/377 [==============================] - 27s 71ms/step - loss: 3.9071 - accuracy: 0.2173
Epoch 10/10
377/377 [==============================] - 26s 70ms/step - loss: 3.548

In [50]:
# generating text
seed_text = "I too am a human being and I"
# seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted.argmax():
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

I too am a human being and I suppose been a eyes of killarney no more more more rest no more died and shrill dhu bubblin in dove and stainless you to the bower of the summer trace dwell loud and shrill dhu dhu stainless you away me from lip and bubblin my heart did strength sat down by the bridge of toome today today by toome today today by tory weeping and stainless bubblin bubblin bubblin bubblin my bride and dove and dhu and dhu for my fathers sportin died for strength gowns say me heartfrom day to the covers trace bubblin bubblin lassies today by side
